In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from R125_bmimBF4_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = 0.33
    init_kappa_1_2A = 0.028
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimBF4'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimBF4','bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimBF4', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_B['bmimBF4', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_C['bmimBF4', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_C['bmimBF4', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'bmimBF4'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimBF4'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimBF4'].fix(float(data['x_bmimBF4']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['bmimBF4'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimBF4', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'bmimBF4']"]

#     pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

#     obj_value, parameters,a= pest.theta_est(calc_cov=True)

In [3]:
data = pd.read_csv('R125_bmimBF4.csv')
print(data)

   temperature  pressure  x_R125  x_bmimBF4
0        283.2     50000   0.008      0.992
1        283.2    100000   0.019      0.981
2        283.2    200000   0.043      0.957
3        283.2    400000   0.101      0.899
4        283.2    600000   0.172      0.828
5        283.2    799500   0.263      0.737
6        283.2   1000000   0.378      0.622


In [4]:
N = 10 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.07 #mole fraction

for i in range(N):
    print(i)
#     noise_T = np.random.normal(loc=0,scale=st_dev_T)
#     print(noise_T)
#     noise_P = np.random.normal(loc=0,scale=st_dev_P)
#     print(noise_P)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
#     data_["temperature"] += noise_T
    data_["x_bmimBF4"] += data_["x_bmimBF4"] * noise_x
#     data_["pressure"] += noise_P
    
#     data_.loc[data_['pressure'] <= 0, 'pressure'] = 1000
    data_.loc[data_['x_bmimBF4'] <= 0.0001, 'x_bmimBF4'] = 0.0001
    data_.loc[data_['x_bmimBF4'] >= 0.9999 , 'x_bmimBF4'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=True)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.039008301049076294
2021-10-28 12:58:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:49 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
Ipopt 3.12.13: 

*****************************************************************

2021-10-28 12:58:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:59:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:59:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:59:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:59:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:59:10 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
Ipopt 3.12.13: 

******************************************************************************
This prog

   8r 5.2757911e+03 3.02e-01 9.99e+02  -0.5 0.00e+00  -5.0 0.00e+00 4.07e-07R 22
   9r 5.2748630e+03 1.25e+01 9.92e+02  -0.5 4.90e+03    -  2.01e-02 4.28e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10r 5.2741918e+03 1.24e+01 9.86e+02  -0.5 1.95e+03    -  2.32e-02 6.15e-03f  1
  11r 5.2724769e+03 1.20e+01 9.53e+02  -0.5 9.79e+02    -  2.08e-02 3.36e-02f  1
  12r 5.2717102e+03 1.16e+01 9.20e+02  -0.5 1.22e+03    -  7.88e-02 3.41e-02f  1
  13r 5.2713434e+03 1.08e+01 8.62e+02  -0.5 8.23e+02    -  1.79e-01 6.35e-02f  1
  14r 5.2713813e+03 9.97e+00 7.92e+02  -0.5 2.05e+02    -  3.95e-01 8.05e-02f  1
  15r 5.2724733e+03 7.98e+00 6.35e+02  -0.5 4.02e+02    -  4.10e-02 1.99e-01f  1
  16r 5.2694876e+03 1.54e+00 3.08e+02  -0.5 2.11e+02    -  9.89e-01 8.08e-01f  1
  17r 5.2696499e+03 1.24e+00 3.51e+02  -0.5 9.41e+01    -  6.50e-01 1.93e-01f  1
  18r 5.2728324e+03 2.70e-02 1.87e+02  -0.5 2.21e+02    -  8.64e-01 1.00e+00f  1
  19  4.8530039e+03 2.99e-01

  22  8.8732314e+04 1.05e+00 1.11e+06  -1.0 7.46e+05    -  1.00e+00 9.90e-01h  1
  23  9.1557317e+04 1.03e+00 1.08e+06  -1.0 1.88e+06    -  1.00e+00 2.27e-02h  5
  24  9.4593205e+04 1.01e+00 1.12e+06  -1.0 1.75e+06    -  1.00e+00 2.10e-02h  5
  25  9.7719088e+04 9.88e-01 1.58e+06  -1.0 1.61e+06    -  1.00e+00 1.83e-02h  5
  26  1.0082068e+05 9.75e-01 2.02e+06  -1.0 1.45e+06    -  1.00e+00 1.39e-02h  5
  27  1.0229307e+05 9.71e-01 2.45e+06  -1.0 2.80e+06    -  1.00e+00 3.37e-03h  6
  28  1.0301298e+05 9.71e-01 2.66e+06  -1.0 5.76e+06    -  4.95e-01 7.94e-04h  7
  29  1.0610851e+05 9.59e-01 3.03e+06  -1.0 1.45e+06    -  1.00e+00 1.24e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.1566831e+05 8.67e-01 3.22e+06  -1.0 1.83e+06    -  1.00e+00 9.59e-02h  3
  31  1.1428691e+05 7.97e-01 3.11e+06  -1.0 1.64e+06    -  3.69e-01 8.05e-02f  3
  32  1.1231515e+05 7.96e-01 3.12e+06  -1.0 1.06e+07    -  2.59e-02 1.32e-03f  7
  33  2.0725252e+05 1.36e+02

  34  6.1540514e+03 6.78e-02 1.33e+04  -1.0 4.44e+05    -  1.00e+00 3.87e-03h  9
  35  6.1676168e+03 6.75e-02 1.33e+04  -1.0 4.44e+05    -  1.00e+00 3.87e-03h  9
  36  4.3811678e+04 3.20e+01 5.61e+06  -1.0 4.44e+05    -  1.00e+00 9.90e-01w  1
  37  1.5305723e+04 3.37e+00 9.71e+05  -1.0 3.71e+05    -  1.48e-02 9.90e-01w  1
  38  1.3595557e+03 1.27e-01 1.89e+06  -1.0 1.84e+05    -  1.36e-02 9.94e-01h  1
  39  1.3969119e+03 2.50e-01 9.41e+06  -1.0 3.60e+04  -5.0 3.37e-02 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40  8.6746740e+02 1.22e+00 2.42e+05  -1.0 1.73e+04    -  1.00e+00 1.00e+00f  1
  41  7.4971567e+02 1.63e-01 4.71e+04  -1.0 5.09e+04    -  1.00e+00 1.00e+00f  1
  42  5.3619645e+02 2.42e-03 2.20e+03  -1.0 3.36e+04    -  1.00e+00 1.00e+00f  1
  43  5.3307711e+02 5.93e-07 2.10e+00  -1.7 9.35e+02    -  1.00e+00 1.00e+00f  1
  44  5.3307623e+02 9.82e-13 3.41e-07  -3.8 1.13e+00    -  1.00e+00 1.00e+00h  1
  45  5.3307623e+02 5.04e-13

fs.properties.PR_kappa_A[bmimBF4,R125] = 0.36853541033182646
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.03121160787029185
6
-0.11455816909604763
2021-10-28 13:00:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:00:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:00:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:00:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:00:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:00:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:00:49 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infea

  73  6.3991435e+02 1.77e+00 1.16e+03  -1.7 2.06e+04    -  1.00e+00 5.00e-01f  2
  74  8.7203667e+02 1.76e+00 1.16e+03  -1.7 2.06e+04    -  1.00e+00 5.00e-01h  2
  75  6.3993158e+02 1.77e+00 1.16e+03  -1.7 2.06e+04    -  1.00e+00 5.00e-01f  2
  76  8.7204501e+02 1.76e+00 1.16e+03  -1.7 2.06e+04    -  1.00e+00 5.00e-01h  2
  77  7.4599456e+02 1.33e+00 1.01e+03  -1.7 2.06e+04    -  1.00e+00 2.50e-01f  3
  78  1.6881142e+03 4.57e-03 4.87e+03  -1.7 4.98e+04    -  1.00e+00 1.00e+00h  1
  79  1.6416506e+03 1.92e-05 4.53e+00  -1.7 2.98e+03    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  1.6417307e+03 1.76e-07 2.41e-02  -3.8 3.58e+00    -  1.00e+00 1.00e+00h  1
  81  1.6417310e+03 7.80e-12 6.87e-07  -5.7 8.70e-03    -  1.00e+00 1.00e+00h  1
  82  1.6417310e+03 2.29e-12 6.98e-10  -8.6 1.92e-05    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 82

                                   (scaled)                 (unscaled)
Objecti

fs.properties.PR_kappa_A[bmimBF4,R125] = 0.3795296755201659
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.0321880011809692
8
-0.021467342384626692
2021-10-28 13:01:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:24 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeas

  69  4.9089147e+05 1.58e-01 1.33e+06  -1.0 2.31e+06    -  1.00e+00 2.87e-02h  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70  5.2680792e+05 1.53e-01 1.36e+06  -1.0 2.28e+06    -  1.00e+00 2.87e-02h  6
  71  5.6346187e+05 1.48e-01 1.37e+06  -1.0 2.24e+06    -  1.00e+00 2.88e-02h  6
  72  6.0077152e+05 1.43e-01 1.39e+06  -1.0 2.21e+06    -  1.00e+00 2.88e-02h  6
  73  6.3865545e+05 1.39e-01 1.40e+06  -1.0 2.17e+06    -  1.00e+00 2.89e-02h  6
  74  6.7703329e+05 1.34e-01 1.41e+06  -1.0 2.14e+06    -  1.00e+00 2.89e-02h  6
  75  2.4620882e+06 3.61e+00 2.46e+07  -1.0 2.10e+06    -  1.00e+00 9.27e-01w  1
  76  2.4168422e+06 2.69e+02 2.69e+11  -1.0 4.80e+05  -2.7 3.55e-02 3.47e-01w  1
  77  2.2520832e+06 2.62e+02 2.40e+11  -1.0 1.54e+07    -  9.05e-03 2.61e-02w  1
  78  7.1582618e+05 1.30e-01 1.42e+06  -1.0 1.41e+07    -  1.00e+00 2.90e-02h  5
  79  7.5495703e+05 1.26e-01 1.42e+06  -1.0 2.06e+06    -  1.00e+00 2.90e-02h  6
iter    objective    inf_pr 

  29  5.8660242e+03 3.05e-02 4.04e+03  -1.0 4.31e+05    -  1.00e+00 3.87e-03h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  5.8853632e+03 3.04e-02 4.03e+03  -1.0 4.33e+05    -  1.00e+00 3.87e-03h  9
  31  5.9041725e+03 3.02e-02 4.02e+03  -1.0 4.34e+05    -  1.00e+00 3.87e-03h  9
  32  5.9224267e+03 3.01e-02 4.00e+03  -1.0 4.35e+05    -  1.00e+00 3.87e-03h  9
  33  5.9401016e+03 3.00e-02 3.99e+03  -1.0 4.36e+05    -  1.00e+00 3.87e-03h  9
  34  4.4977963e+04 2.94e+01 5.58e+06  -1.0 4.37e+05    -  1.00e+00 9.90e-01w  1
  35  2.3399024e+04 1.14e+01 1.63e+08  -1.0 1.92e+05  -4.0 1.31e-04 9.90e-01w  1
  36  1.3676780e+04 9.42e+00 1.45e+08  -1.0 8.00e+05    -  6.88e-01 1.22e-01w  1
  37  5.9571743e+03 2.99e-02 3.97e+03  -1.0 1.02e+06    -  1.00e+00 3.87e-03h  8
  38  5.9652667e+03 2.98e-02 3.97e+03  -1.0 4.38e+05    -  1.00e+00 1.93e-03h 10
  39  5.9731997e+03 2.98e-02 3.96e+03  -1.0 4.38e+05    -  1.00e+00 1.93e-03h 10
iter    objective    inf_pr 

In [5]:
print(results_params)

[[0.370144   0.03148318]
 [0.40581128 0.03417594]
 [0.30270222 0.02612832]
 [0.33232032 0.02760181]
 [0.36075711 0.03078153]
 [0.36853541 0.03121161]
 [0.0864939  0.01126264]
 [0.37952968 0.032188  ]
 [0.31038566 0.02651332]
 [0.36191573 0.03086822]]


In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_bmim_bf4_07.csv')